In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='sigmoid')(x)

In [ ]:
vgg16_model = Model(inputs=base_model.input, outputs=x)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
vgg16_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_dataset = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'C:/Users/Aditi Bolakhe/Desktop/drowsy detection/cnn1/training',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_dataset = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'C:/Users/Aditi Bolakhe/Desktop/drowsy detection/cnn1/valid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
val_dataset.class_indices

In [ ]:
print(f"Input shape: {train_dataset.image_shape}")
print(f"Output shape: {vgg16_model.output_shape}")

In [ ]:
model_fit = vgg16_model.fit(train_dataset,
                      epochs=10,
                      validation_data=val_dataset)

In [ ]:
dir_path ='C:/Users/Aditi Bolakhe/Desktop/cnn1/testing'

for i in os.listdir(dir_path):
    img = image.load_img(dir_path+ '//'+ i, target_size=(224,224))
    plt.imshow(img)
    plt.show()
    
    
    x=image.img_to_array(img)
    x=np.expand_dims(x,axis=0)
    images=np.vstack([x])
    
    val =vgg16_model.predict(images)
    print(val)
    if (val == 1.0).all():
         print("you are active")
    elif (val >= 0.1).all() :
         print("you are drowsy")
    else:
      print("you are active")

In [ ]:
scores = vgg16_model.evaluate(val_dataset, verbose=0)
print(f"Test loss: {scores[0]}")
print(f"Test accuracy: {scores[1]}")